In [5]:
# import
import re
import pandas as pd
import numpy as np

import email
from email.parser import Parser


import unicodedata

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

import acquire
from prepare import basic_clean, tokenize, stem, lemmatize, remove_stopwords

In [ ]:
df = pd.read_csv('emails.csv')

In [ ]:
df.head()

In [ ]:
df.message[0]

In [ ]:
import re
re.findall(r'\n\n.*\n', df.message[0])

In [ ]:
import email
from email.parser import Parser

In [ ]:
message = df.message[0]

In [ ]:
headers = Parser().parsestr(message)
headers

In [ ]:
print(headers)

In [ ]:
type(headers)

In [ ]:
print(headers['From'])

In [ ]:
print(headers['Date'])

In [ ]:
print(headers['To'])

In [ ]:
print(headers['Message-ID'])

In [ ]:
headers.get_payload()

In [ ]:
bodies = []
dates = [] 
for i in df.message:
    headers = Parser().parsestr(i)
    body = headers.get_payload()
    date = headers['Date']
    bodies.append(body)
    dates.append(date)

In [ ]:
# bodies

In [ ]:
len(bodies)

In [ ]:
# dates

In [ ]:
len(dates)

In [ ]:
bodies

In [ ]:
body_df = pd.DataFrame(bodies, columns = ['content'])
dates_df = pd.DataFrame(dates, columns = ['date'])

In [ ]:
body_df.head()

In [ ]:
dates_df.head()

In [ ]:
df['content'] = body_df

In [ ]:
df['date'] = dates_df

In [ ]:
df.head()

In [ ]:
from_emails = []
to_emails = []
subjects = []
for i in df.message:
    headers = Parser().parsestr(i)
    from_email = headers['From']
    to_email = headers['To']
    subject = headers['Subject']
    from_emails.append(from_email)
    to_emails.append(to_email)
    subjects.append(subject)

In [ ]:
from_email_df = pd.DataFrame(from_emails, columns = ['from_email'])
to_email_df = pd.DataFrame(to_emails, columns = ['to_email'])
subjects_df = pd.DataFrame(subjects, columns = ['subject'])

In [ ]:
from_email_df.head()

In [ ]:
to_email_df.head()

In [ ]:
subjects_df.head()

In [ ]:
df['from_email'] = from_email_df
df['to_email'] = to_email_df
df['subject'] = subjects_df


In [ ]:
df.head()

In [ ]:
df.message[2]

In [ ]:
df.file[0]

In [ ]:
df.file[1]

In [ ]:
df.file[1000]

In [ ]:
df.file[50000]

In [ ]:
df.file[100000]

In [ ]:
# Defining Acquire 

In [ ]:
df.head()

In [ ]:
def acquire_email():
    'This functions will get us contents of email and from email, to email and subject'
    
    # pandas helping us to read csv file
    df = pd.read_csv('emails.csv')
    
    # empty list for keeping contents
    bodies = []
    
    # empty list for keeping dates
    dates = []
    
    # empty list for keeping emails from
    from_emails = []
    
    # empty list for keeping emails to
    to_emails = []
    
    # empty list for keeping subjects from
    subjects = []
    
    # loop to add content, date, From, To, Subject in an empty list
    for i in df.message:
        headers = Parser().parsestr(i)
        body = headers.get_payload()
        date = headers['Date']
        from_email = headers['From']
        to_email = headers['To']
        subject = headers['Subject']
        bodies.append(body)
        dates.append(date)
        from_emails.append(from_email)
        to_emails.append(to_email)
        subjects.append(subject)
            
    # making DatFrame out of list created by lopp
    body_df = pd.DataFrame(bodies, columns = ['content'])
    dates_df = pd.DataFrame(dates, columns = ['date'])
    from_email_df = pd.DataFrame(from_emails, columns = ['from_email'])
    to_email_df = pd.DataFrame(to_emails, columns = ['to_email'])
    subjects_df = pd.DataFrame(subjects, columns = ['subject'])
    
    # giving a name to the column created in DataFrame
    df['content'] = body_df
    df['date'] = dates_df
    df['from_email'] = from_email_df
    df['to_email'] = to_email_df
    df['subject'] = subjects_df
    
    return df
    

In [ ]:
df= acquire_email()
df

In [ ]:
df_needed = df.drop(columns = ['file', 'message', 'from_email', 'to_email', 'subject'])

In [ ]:
df_needed.head()

In [ ]:
df_needed.to_csv('content_file')

In [2]:
df_new =pd.read_csv('content_file')

In [3]:
df_new.head()

,Unnamed: 0,content,date
0,0,Here is our forecast\n\n,"Mon, 14 May 2001 16:39:00 -0700 (PDT)"
1,1,Traveling to have a business meeting takes the...,"Fri, 4 May 2001 13:51:00 -0700 (PDT)"
2,2,test successful. way to go!!!,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)"
3,3,"Randy,\n\n Can you send me a schedule of the s...","Mon, 23 Oct 2000 06:13:00 -0700 (PDT)"
4,4,Let's shoot for Tuesday at 11:45.,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)"


In [ ]:
import acquire

In [ ]:
acquire.acquire_email()

In [ ]:
df_needed.head()

# Prepare

In [ ]:
import prepare

In [ ]:
df_new.content[0]

In [ ]:
def basic_clean(s):
    '''
    Takes a string and returns a normalized lowercase string 
    with special characters removed
    '''
    # droping \n
    s = s.replace('\n', ' ')
    s = s.replace('\t', ' ')
   
    # strip
    s = s.strip()
    # lowercase
    s = str(s.lower())
    # normalize
    s = unicodedata.normalize('NFKD', s)\
    .encode('ascii', 'ignore')\
    .decode('utf-8', 'ignore')
    # remove special characters and lowercase
    s = re.sub(r"[^a-z0-9'\s]", '', s)
    
    return s

In [ ]:
basic_clean(df_new.content[0])

In [ ]:
basic_clean(df_new.content[1])

In [ ]:
basic_clean(df_new.content[2])

In [ ]:
basic_clean

In [ ]:
def tokenize(s):
    '''
    Takes a string and returns a tokenized version of the string
    '''
    # create tokenizer
    tokenizer = nltk.tokenize.ToktokTokenizer()
    # return tokenized string
    return tokenizer.tokenize(s, return_str=True)

In [ ]:
tokenize(df_new.content[0])

In [ ]:
tokenize(df_new.content[1])

In [4]:
from prepare import basic_clean, tokenize, stem, lemmatize, remove_stopwords

In [ ]:
def prep_string_data(df, column, extra_words=[], exclude_words=[]):
    '''
    Takes in a dataframe, original string column, with optional lists of words to
    add to and remove from the stopword_list. Returns a dataframe with the title,
    original column, and clean, stemmed, and lemmatized versions of the column.
    '''
    df['clean'] = df[column].apply(basic_clean).apply(tokenize).apply(remove_stopwords, extra_words=extra_words, exclude_words=exclude_words)
    
    df['stemmed'] = df['clean'].apply(tokenize).apply(stem)

    df['lemmatized'] = df['clean'].apply(tokenize).apply(lemmatize)

    
    return df[['title', column, 'clean', 'stemmed', 'lemmatized']]

In [6]:
def prep_article_data(df, column, extra_words=[], exclude_words=[]):
    '''
    This function take in a df and the string name for a text column with 
    option to pass lists for extra_words and exclude_words and
    returns a df with the text article title, original text, stemmed text,
    lemmatized text, cleaned, tokenized, & lemmatized text with stopwords removed.
    '''
    df['clean'] = df[column].apply(basic_clean)\
                            .apply(tokenize)\
                            .apply(remove_stopwords, 
                                   extra_words=extra_words, 
                                   exclude_words=exclude_words)
    
    df['stemmed'] = df[column].apply(basic_clean)\
                            .apply(tokenize)\
                            .apply(stem)\
                            .apply(remove_stopwords, 
                                   extra_words=extra_words, 
                                   exclude_words=exclude_words)
    
    df['lemmatized'] = df[column].apply(basic_clean)\
                            .apply(tokenize)\
                            .apply(lemmatize)\
                            .apply(remove_stopwords, 
                                   extra_words=extra_words, 
                                   exclude_words=exclude_words)
    
    return df[['date', column,'clean', 'stemmed', 'lemmatized']]

In [7]:
df_new = prep_article_data(df_new, 'content')
df_new

,date,content,clean,stemmed,lemmatized
0,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",Here is our forecast\n\n,forecast,forecast,forecast
1,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",Traveling to have a business meeting takes the...,traveling business meeting takes fun trip espe...,travel busi meet take fun trip especi prepar p...,traveling business meeting take fun trip espec...
2,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",test successful. way to go!!!,test successful way go,test success way go,test successful way go
3,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)","Randy,\n\n Can you send me a schedule of the s...",randy send schedule salary level everyone sche...,randi send schedul salari level everyon schedu...,randy send schedule salary level everyone sche...
4,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",Let's shoot for Tuesday at 11:45.,let ' shoot tuesday 1145,let ' shoot tuesday 1145,let ' shoot tuesday 1145
...,...,...,...,...,...
517396,"Wed, 28 Nov 2001 13:30:11 -0800 (PST)",This is a trade with OIL-SPEC-HEDGE-NG (John L...,trade oilspechedgeng john lavorato ' book john...,thi trade oilspechedgeng john lavorato ' book ...,trade oilspechedgeng john lavorato ' book john...
517397,"Wed, 28 Nov 2001 12:47:48 -0800 (PST)",Some of my position is with the Alberta Term b...,position alberta term book send positions dire...,posit alberta term book send onli posit direct...,position alberta term book send position direc...
517398,"Wed, 28 Nov 2001 07:20:00 -0800 (PST)",2\n\n -----Original Message-----\nFrom: \tDouc...,2 original message doucet dawn sent wednesday ...,2 origin messag doucet dawn sent wednesday nov...,2 original message doucet dawn sent wednesday ...
517399,"Tue, 27 Nov 2001 11:52:45 -0800 (PST)",Analyst\t\t\t\t\tRank\n\nStephane Brodeur\t\t\...,analyst rank stephane brodeur 1 chad clark 1 i...,analyst rank stephan brodeur 1 chad clark 1 ia...,analyst rank stephane brodeur 1 chad clark 1 i...


In [8]:
# df_new.to_csv('all')

In [ ]:
# df_new['content'].apply(basic_clean).apply(tokenize).apply(remove_stopwords, extra_words=extra_words, exclude_words=exclude_words)

In [ ]:
# df df_new['content'].apply(basic_clean).apply(tokenize).apply(remove_stopwords)
  

In [9]:
df_new['lemmatized'].head(100)

0                                              forecast
1     traveling business meeting take fun trip espec...
2                                test successful way go
3     randy send schedule salary level everyone sche...
4                              let ' shoot tuesday 1145
                            ...                        
95    mac thanks research report eog observation gas...
96    forwarded phillip k allenhouect 08202000 0539 ...
97    forwarded phillip k allenhouect 08202000 0538 ...
98    forwarded phillip k allenhouect 08202000 0538 ...
99    checked exercising option smith barney enron h...
Name: lemmatized, Length: 100, dtype: object

In [ ]:
# prep_string_data(df_new, df_new.content)